# Working with Sea Ice Concentration Data

> history \| Updated October 2023 <br/>

Most of the satellite data that PolarWatch distributes are gridded data in raster format. Gridded data is a data structure comprising cells arranged in rows and columns to form a grid. Each grid cell has a unique location, defined by coordinates (latitude and longitude, or x and y), and contains values such as measurements (e.g. microwave brightness temperatures) or geophysical property data (e.g. sea ice concentration).


## NOAA/NSIDC Sea Ice Concentration Data

NOAA-funded NSIDC Sea Ice Concentration Climate Data Record on PolarWatch are gridded. The data are projected using the <strong><a href="https://nsidc.org/data/user-resources/help-center/guide-nsidcs-polar-stereographic-projection" target="_blank">Polar Stereographic Projection</a></strong> in x and y coordinates.  There are two NOAA/NSIDC Sea Ice Concentration data distributed on PolarWatch (G02202, G10016).  

*  Ice Concentration NSIDC Climate Data Record V4 (G02202),  daily and monthly
*  Ice Concentration NSIDC Near-Real-Time (NRT) Climate Data Record V2 (G10016), daily and monthly


__Userful Links__
* Data Information at NSIDC <a href="https://nsidc.org/data/g02202/versions/4">Climate Data Record V4 (G02202)</a>,   <a href="https://nsidc.org/data/g10016/versions/2">Near Real Time V2 (G10016)</a>
* Data Download at <a href="https://polarwatch.noaa.gov/catalog/">NOAA PolarWatch Data Catalog</a> (search for sea ice)


    
## Code sample

This notebook includes python code sample demonstrating the following:

* Downloading sea ice concentration data in netcdf file format (data download, data import)
* Accessing information about the data (metadata)
* Accessing sea ice concentration data

## Dataset used


__Sea Ice Concentration, NOAA/NSIDC Climate Data Record V4, Northern Hemisphere, 25km, Science Quality, 1978-Present, Monthly__  
This dataset includes sea ice concentration data from the northern hemisphere, and is produced by the NOAA/NSIDC using the Climate Data Record algorithm.  The resolution is 25km, meaning each grid in this data set represents a value that covers a 25km by 25km area.  

This data can be previewed at the NOAA PolarWatch Data Portal: <a href="https://polarwatch.noaa.gov/catalog/ice-sq-nh-nsidc-cdr-v4/preview/?dataset=daily&var=cdr_seaice_conc&time_min=2022-12-31T00:00:00Z&time_max=2022-12-31T00:00:00Z&proj=epsg3413&colorBar=KT_ice|||0|1|">Sea Ice Concentration NSIDC Climate Data Record V4, Arctic Page</a>.



## Download Data

### Download from web interface

There are few ways to preview and download data from PolarWatch webpage
- PolarWatch ERDDAP: Data distribution system with data preview, subset and download features (<a href="https://polarwatch.noaa.gov/erddap/griddap/nsidcG02202v4nhmday.html">Link</a>)
- PolarWatch Portal: User friendly interface to the ERDDAP (<a href="https://polarwatch.noaa.gov/catalog/ice-sq-nh-nsidc-cdr-v4/preview/">Link</a>)

### Download using API
Data can be obtained by sending a data request with an URL that includes data information.

__How to create the url?__
* Go to the PolarWatch data preview page <a href="https://polarwatch.noaa.gov/catalog/ice-sq-nh-nsidc-cdr-v4/preview/">https://polarwatch.noaa.gov/catalog/ice-sq-nh-nsidc-cdr-v4/preview/ </a>
* Click on <strong>Quick Download</strong> button
* Enter the time and area of interest
* Select NetCDF as a File format (default is already set to netCDF)
* Click <strong>View Url</strong>
* or URL can be independently constructed (next section)

## Download and Access Data with url

<strong>```xarray```</strong> package will be used to work with gridded dataset and must be downloaded and installed to run the notebook.  To learn more about ```xarray```, visit <a href="https://docs.xarray.dev/en/stable/index.html">xarray documentation page</a>.   

In [1]:
# Load required modules
import urllib.request # for url data request
import xarray as xr # for gridded data
import datetime as dt

In [2]:
# @title
# Construct PolarWatch Data Request URL

# Set PolarWatch erddap address
base_url = 'https://polarwatch.noaa.gov/erddap/griddap/'

# Set erddap datasetID for noaa/nsidc G02202 sea ice concentration data
datasetID = 'nsidcG02202v4nhmday'

# Set file format (netcdf)
file_type = '.nc'
query_start = '?'

# Set variable name of interest
variable_name = 'cdr_seaice_conc_monthly'

# Set date range
date_range = '[(2023-06-01T00:00:00Z):1:(2023-06-01T00:00:00Z)]'

year, month, day = 2023, 6, 1
date_start = dt.datetime(year, month, day)

year, month, day = 2023, 6, 1
date_end = dt.datetime(year, month, day)
date_range = '[('+
              date_start.strftime("%Y-%m-%dT%H:%M:%SZ")+
              ':1:'+
              date_end.strftime("%Y-%m-%dT%H:%M:%SZ") +
              ')]'

# Set spatial range
spatial_range = '[(4843696.04):1:(-4858210.64)][(-3850000.0):1:(3750000.0)]'

# Create data request URL
url = ''.join([base_url,
               datasetID,
               file_type,
               query_start,
               variable_name,
               date_range,
               spatial_range
               ])
# Examine URL
url

SyntaxError: invalid syntax (<ipython-input-2-58a80ae33bdd>, line 25)

In [ ]:
# Send the data request and save the data in a file
urllib.request.urlretrieve(url, "sic.nc")

# Open the netCDF file to create an xarray dataset object
ds = xr.open_dataset("sic.nc")

## Access Metadata (Information about data)

<strong>```xarray```</strong> dataset has key properties describing the data
* values, dims, coords, attrs

In [ ]:
# Examine coordinates and their variable names (time, ygrid, xgrid)
# to access each coordinate, example: ds.coords['time']
ds.coords

<xarray.DataArray 'time' ()>
array('2023-06-01T00:00:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2023-06-01
Attributes:
    _ChunkSizes:          1024
    _CoordinateAxisType:  Time
    actual_range:         [1.6855776e+09 1.6855776e+09]
    axis:                 T
    ioos_category:        Time
    long_name:            ANSI date
    standard_name:        time
    time_origin:          01-JAN-1970 00:00:00

In [ ]:
# Examine dimensions of coordinates
# access dim of each coordinate: ds.dims['time']
ds.dims

Frozen({'time': 1, 'ygrid': 389, 'xgrid': 304})

In [ ]:
# Examine all attributes associated with the dataset
# access specific attribute, ex. ds.attrs['grid_mapping_proj4text']
ds.attrs

In [ ]:
# Examine names of variables
list(ds.keys())

['cdr_seaice_conc_monthly']

In [ ]:
# Examine information about variables
ds.keys()

KeysView(<xarray.Dataset>
Dimensions:                  (time: 1, ygrid: 389, xgrid: 304)
Coordinates:
  * time                     (time) datetime64[ns] 2023-06-01
  * ygrid                    (ygrid) float32 4.838e+06 4.812e+06 ... -4.862e+06
  * xgrid                    (xgrid) float32 -3.838e+06 -3.812e+06 ... 3.738e+06
Data variables:
    cdr_seaice_conc_monthly  (time, ygrid, xgrid) float32 ...
Attributes: (12/65)
    acknowledgement:                                     This project was sup...
    cdm_data_type:                                       Grid
    cdr_variable:                                        cdr_seaice_conc_monthly
    comment:                                             none
    contributor_name:                                    Walter N. Meier, Flo...
    contributor_role:                                    principal investigat...
    ...                                                  ...
    summary:                                             This data s

## Access Variable Data

```ds.keys()``` and ```list(ds.keys())``` from the previous cells show the variable name: ```cdr_seaice_conc_monthly```.

In [ ]:
# Examine variable cdr_seaice_conc_monthly
ds['cdr_seaice_conc_monthly']

<xarray.DataArray 'cdr_seaice_conc_monthly' (time: 1, ygrid: 389, xgrid: 304)>
[118256 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2023-06-01
  * ygrid    (ygrid) float32 4.838e+06 4.812e+06 ... -4.838e+06 -4.862e+06
  * xgrid    (xgrid) float32 -3.838e+06 -3.812e+06 ... 3.712e+06 3.738e+06
Attributes: (12/13)
    ancillary_variables:  stdev_of_cdr_seaice_conc_monthly qa_of_cdr_seaice_c...
    colorBarMaximum:      1.0
    colorBarMinimum:      0.0
    colorBarPalette:      KT_ice
    datum:                +ellps=urn:ogc:def:crs:EPSG::4326
    flag_meanings:        pole_hole lakes coastal land_mask missing_data
    ...                   ...
    ioos_category:        Ice Distribution
    long_name:            NOAA/NSIDC Climate Data Record of Passive Microwave...
    references:           https://nsidc.org/data/g02202/versions/4/
    standard_name:        sea_ice_area_fraction
    units:                1
    valid_range:          [0. 1.]

In [ ]:
# gridded data shape describes the dimensions of the  variable (time, ygrid, xgrid)
ds['cdr_seaice_conc_monthly'].shape


(1, 389, 304)

## More sea ice code samples

This code sample demonstrated how to obtain and examine the gridded sea ice concentration dataset. You can find additional resources at <a href="https://github.com/coastwatch-training/CoastWatch-Tutorials">CoastWatch Tutorial</a>

__Sea Ice Examples__
* <a href="https://github.com/coastwatch-training/CoastWatch-Tutorials/tree/main/transform-to-another-map-projection">calculate-seaice-extent</a> View sea ice concentration (SIC) data on a map with the polar stereographic projection. Calculate and compare sea ice area/extent from multi-year SIC datasets.
    
* <a href="https://github.com/coastwatch-training/CoastWatch-Tutorials/tree/main/transform-to-another-map-projection">transform-to-another-map-projection</a> Access satellite data with polar stereographic coordinates and transform it into a different coordinate system using EPSG code.
      
* <a href="https://github.com/coastwatch-training/CoastWatch-Tutorials/tree/main/map-data-with-different-projections"> map-data-with-different-projections</a> Download and examine a polar stereographic projected dataset, plot the data on a projected map. Add animal track data with geographical coordinates onto the projected map.
    
### Questions and comments

For questions and comments, you can create an issue in the repository or email at polar.watch\@noaa.gov

